https://github.com/abetlen/llama-cpp-python

In [ ]:
!pip install huggingface_hub
!pip install llama-cpp-python

In [ ]:
!pip install --quiet datasets evaluate
!pip install --quiet transformers==4.31.0
!pip install --quiet accelerate==0.21.0

In [ ]:
device_map = {"": 0}
new_model = 'foobar8675/llama-2-7b-sentiment-classifier-GGML'

In [ ]:
from datasets import load_dataset

imdb = load_dataset("imdb")
imdb = imdb.shuffle(seed=42)
dataset = imdb['train'].train_test_split(test_size=0.01, shuffle=False)

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
!CT_CUBLAS=1 pip install ctransformers --no-binary ctransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.1/376.1 kB 5.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for ctransformers: filename=ctransformers-0.2.27-cp310-cp310-linux_x86_64.whl size=2338904 sha256=5a965066e59e7d883eda6f370b103a9e9ddd1c8c9461c39e83c1aae5a6c02ec0
  Stored in directory: /root/.cache/pip/wheels/dd/54/e9/32364da8eee84a2b0b412394983c15add18816c507e90f02d8
Successfully built ctransformers


In [ ]:
!git lfs install
!git clone https://huggingface.co/foobar8675/llama-2-7b-sentiment-classifier-GGML

Git LFS initialized.
Cloning into 'llama-2-7b-sentiment-classifier-GGML'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 12 (delta 2), reused 0 (delta 0), pack-reused 3
Unpacking objects: 100% (12/12), 1.60 KiB | 821.00 KiB/s, done.


In [ ]:
from llama_cpp import Llama

llm = Llama(model_path="llama-2-7b-sentiment-classifier-GGML/ggml-model-q4_0-v2.gguf", n_ctx=2048)
output = llm("Q: Name the planets in the solar system? A: ", max_tokens=2048, stop=["Q:", "\n"], echo=True)
print(output)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


{'id': 'cmpl-947460a8-fac8-4143-b770-2d6e3fd82747', 'object': 'text_completion', 'created': 1696206944, 'model': 'llama-2-7b-sentiment-classifier-GGML/ggml-model-q4_0-v2.gguf', 'choices': [{'text': 'Q: Name the planets in the solar system? A: 1.➖ Mercury 2️⃣️ Venus 3️⃣️ Earth 4️⃣️ Mars 5️⃣️ Jupiter 6️⃣️ Saturn 7️⃣️ Uranus 8️⃣️ Neptune 9️⃣️ Pluto (note: since Pluto is no longer considered a planet, this numbering system does not apply) ', 'index': 0, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 15, 'completion_tokens': 100, 'total_tokens': 115}}


In [ ]:
ds = dataset['test'].select(range(10))

In [ ]:
def format_dolly(sample):
    sys_prompt = """<<SYS>>You are a helpful, respectful and honest assistant. Always answer as helpfully as possible using the context text provided. Your answers should only answer the question once and not have any text after the answer is done.

    If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.<</SYS>>"""
    instruction = f"### Instruction\nRate the sentiment of the following text, using a 0 or 1. 1 is a good sentiment and 0 is a bad one. Do not respond with anything other than a 0 or 1."
    context = f"### Context\n{sample['text']}"
    response = f"### Answer\n"
    prompt = "\n\n".join([i for i in [sys_prompt, instruction, context, response] if i is not None])
    return prompt

from random import randrange
print(format_dolly(dataset["train"][randrange(len(dataset["train"]))]))

<<SYS>>You are a helpful, respectful and honest assistant. Always answer as helpfully as possible using the context text provided. Your answers should only answer the question once and not have any text after the answer is done.

    If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.<</SYS>>

### Instruction
Rate the sentiment of the following text, using a 0 or 1. 1 is a good sentiment and 0 is a bad one. Do not respond with anything other than a 0 or 1.

### Context
This movie surprised me, it had good one-liners and laughs, + a nonstop action-packed storyline with tons of gun action and explosions. This movie surprisingly had a lot of good twists and turns. The plot is solid despite what others may think, it kept my interest the whole time right up till the very end. In conclusion; this is a great way for an action movie buff to 

In [ ]:
import evaluate
import time
from transformers import (
    pipeline
)
accuracy = evaluate.load("accuracy")

refs = []
preds = []
skipped = []
timings = []
iterable_dataset = ds.to_iterable_dataset()
for i, sample in enumerate(iterable_dataset):
    start = time.time()
    answer = llm(format_dolly(sample))
    answer = answer['choices'][0]['text']
    answer = answer[0]
    end = time.time()
    print(answer)
    if answer is None:
        skipped.append(i)
        print("skipped")
    else:
        preds.append(int(answer))
        refs.append(int(sample['label']))
        timings.append(end-start)

Llama.generate: prefix-match hit


1


Llama.generate: prefix-match hit


0


Llama.generate: prefix-match hit


...

0


Llama.generate: prefix-match hit


0


Llama.generate: prefix-match hit


1


In [ ]:
accuracy.add_batch(references=refs, predictions=preds)
accuracy.compute()

{'accuracy': 0.9}

In [ ]:
import numpy as np
print(f"{round(np.average(timings),2)} seconds per report")

97.76 seconds per report


In [ ]:
from transformers import pipeline
import evaluate
accuracy = evaluate.load("accuracy")
refs = []
preds = []
durations = []
iterable_dataset = ds.to_iterable_dataset()
for sample in iterable_dataset:
    start = time.time()
    classifier = pipeline("sentiment-analysis", model="siebert/sentiment-roberta-large-english", truncation=True)
    duration = time.time()-start
    result = classifier(sample["text"])
    preds.append(int(result[0]['label'] == 'POSITIVE'))
    refs.append(int(sample['label']))
    durations.append(duration)

accuracy.add_batch(references=refs, predictions=preds)
print(f"{accuracy.compute()}, {round(np.average(durations),2)}s")

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


{'accuracy': 1.0}, 7.03s
